# Chatbot

### Google Drive Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Load Quora QA data

In [ ]:
import csv
import numpy as np
import pandas as pd

# Load QA dataset
with open("/content/sample_data/quora_all_qa.csv", 'r') as file:
  csvreader = csv.reader(file)
  question_list = []
  answer_list = []
  for row in csvreader:
    question_list.append(row[0])
    answer_list.append(row[1])
    
crypto_df = pd.DataFrame()
crypto_df['question'] = question_list
crypto_df['answer'] = answer_list
copy_question_list = question_list.copy()

crypto_df

,question,answer
0,What is the best crypto wallet between Coinbas...,Binance and Coinbase are two cryptocurrency pl...
1,Is Binance trustworthy,Binance is the world's largest cryptocurrency ...
2,How do I create an account on Binance,Binance is still the largest crypto-to-crypto ...
3,Which is a better exchange for Indians to trad...,Running a cryptocurrency business in India is ...
4,What are the best coins to invest in binance com,"You can try to have a look on lumi coin, peopl..."
...,...,...
16831,Whats your opinion about the future of cryptoc...,I am going to try to make it as simple as poss...
16832,What is a good exchange platform to buy Ripple...,"Ah yes, Ripple (XRP) [1] - the most divisive c..."
16833,I am investing 70 of my whole savings in BTC a...,Since Bitcoin made waves in 2017 for hitting a...
16834,Do you think BAT can break into the top 10 in ...,The answer is yes to both. The price is a simp...


### Embedding our questions using SBERT

In [4]:
# Install sentence transformer (https://www.sbert.net/)
!pip install sentence_transformers==2.2.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import random
import torch
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Embedding question sentences
bert_model = SentenceTransformer('sentence-transformers/distilbert-base-nli-max-tokens')
sentence_embeddings = bert_model.encode(question_list)

In [ ]:
import joblib
# Save the embedding
# joblib.dump(sentence_embeddings, '/content/drive/MyDrive/Colab Notebooks/CSCI544_ANLP/data/embedding_distilbert-base-nli-max-tokens.pkl')

# Load the embedding
# sentence_embeddings = joblib.load('/content/drive/MyDrive/Colab Notebooks/CSCI544_ANLP/data/embedding_distilbert-base-nli-max-tokens.pkl')

['/content/drive/MyDrive/Colab Notebooks/CSCI544_ANLP/data/embedding_distilbert-base-nli-max-tokens.pkl']

In [ ]:
# Check an example of cosine-similarity between similar questions
question1 = bert_model.encode(['how do people send crypto from a trust wallet to binance'])
question2 = bert_model.encode(['how to send cyrto to binance from trust wallet'])
print(cosine_similarity(question2,question1))

[[0.92691123]]
[[0.9464959]]


### Define functions

In [ ]:
# There are many versions not working. Make sure to use the below version.
!pip install googletrans==4.0.0-rc1

In [ ]:
from bs4 import BeautifulSoup 
import requests 
import time
from googletrans import Translator

#function to get the price of a cryptocurrency
def get_price(coin):
    
    # make up the URL
    url = "https://www.google.com/search?q=" + coin + "+price"
    
    # make the request
    content = requests.get(url) 
  
    # parse the response
    soup = BeautifulSoup(content.text, 'html.parser') 
  
    # find the price 
    text = soup.find("div", attrs={'class':'BNeawe iBp4i AP7Wnd'}).text
    
    # return the price 
    return text

def translate_question(text, lang):
  # translator = Translator()
  translator = Translator(service_urls=[
        'translate.google.com',
      ])
  
  # If text is not english, translate it.
  return translator.translate(text= text, src=lang, dest='en').text if lang != 'en' else text

def translate_answer(text):
  from googletrans import Translator
  # translator = Translator()
  translator = Translator(service_urls=[
        'translate.google.com',
      ])
  
  lang = translator.detect(text).lang
  
  # If text is not english, translate it.
  return translator.translate(text= text, dest='en').text if lang != 'en' else text

In [ ]:
def return_answer(question):

  ####### Translate question #######
  # detect the language
  translator = Translator(service_urls=[
        'translate.google.com',
      ])
  lang = translator.detect(question).lang

  # If text is not english, translate it.
  if lang != 'en':
    question = translator.translate(text= question, src=lang, dest='en').text
  
  ##########################

  ###### Cryptocurrency price ########
  if 'price' in question.lower():
    # check if user is asking the price of crpytocurrency in our supported list.
    crypto_set = set()
    with open('/content/drive/MyDrive/Colab Notebooks/CSCI544_ANLP/data/top50_crypto.txt') as f_cryto:
      for line in f_cryto.readlines():
        crypto_set.add(line.strip())

    words = question.split()
    coin = ""
    for c in crypto_set:
      _c = c.replace(' ', '').lower()
      if _c in question.replace(' ', '').lower() and len(coin) < len(_c):
        coin = c

    if coin == "":
      return "Sorry, we don't support the given coin."

    return 'The current price of ' + coin + ' is ' + get_price(coin) + "."
  ######################################
  
  ###### Get answer from the question that has highest cosine similarity using SBERT 'distilbert-base-nli-max-tokens' model ######
  question_embedding = bert_model.encode([question])

  scores = cosine_similarity(question_embedding,sentence_embeddings)
  index = np.argmax(cosine_similarity(question_embedding,sentence_embeddings))
  
  # If the cosine similarity is higher or equal to 0.85, return answer
  if scores[0][index] >= 0.85:
    answer = answer_list[index]
  else:
    answer = 'Sorry, I do not understand the question.'

  # If the language of question was not english, translate answer into the given language.
  if lang != 'en':
    answer = translator.translate(text=answer, src='en', dest=lang).text

  return answer


### Chatbot

In [ ]:
while True:
  user_ask = input('Type your question: ')

  if 'bye' in user_ask.lower():
    break

  answer_sentence = return_answer(user_ask)

  print(answer_sentence)
  print('---------------')

  

Type your question: What are the pros and cons of Binance?
Binance's meteoric rise to become the leading cryptocurrency exchange seems incredulous but possible if you know its founder. Changpeng Zhao worked for the Tokyo Stock Exchange and Bloomberg before building one of the fastest high frequency trading systems, Fusion Systems.  Since Binance launched in 2017 in China, its fast execution times have attracted hordes of professional traders. When China shuttered crypto exchanges, Binance moved its offices to Malta. Also, you can request a callback [ https://www.acustomerservice.info/phone-number/binance/ ] from Binance about this.  Binance Jersey and Binance Uganda recently opened to provide fiat-to-crypto trading. With one of the fastest order execution times and institutional-grade research reports, professional crypto traders have found a home on Binance.  Over the counter trading between private parties was introduced in 2019 for trades over 20 BTC. An incubator lab for blockchain

KeyboardInterrupt: ignored

In [ ]:
return_answer('What is the best cryptocurrency?')

What is the best cryptocurrency


"Bitcoin has not just been a trendsetter, ushering in a wave of cryptocurrencies built on a decentralized peer-to-peer network, it's become the de facto standard for cryptocurrencies , inspiring an ever-growing legion of followers and spinoffs.  What is cryptocurrency and altcoin?  Before we take a closer look at some of these alternatives to bitcoin, let's step back and briefly examine what we mean by terms like cryptocurrency and altcoin. A cryptocurrency, broadly defined, is virtual or digital money which takes the form of tokens or coins. While some cryptocurrencies have ventured into the physical world with credit cards or other projects, the large majority remain entirely intangible. The crypto in cryptocurrencies refers to complicated cryptography which allows for a particular digital token to be generated, stored, and transacted securely and, typically, anonymously. Alongside this important crypto feature of these currencies is a common commitment to decentralization; cryptocur